In [7]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from data_loader_saver import load_data, save_data

In [8]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

# List all GPUs
gpus = tf.config.list_physical_devices('GPU')
print("GPUs found:", gpus)

# Quick check
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU available:", len(gpus) > 0)
# Or get the default device name
print("Default GPU device:", tf.test.gpu_device_name())

GPUs found: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: True
GPU available: True
Default GPU device: /device:GPU:0


In [9]:
m_h_datapath = "../data/project/DS-15-samples_mHeights"
n_k_m_datapath = "../data/project/DS-15-samples_n_k_m_P"

In [10]:
n_k_m_P, m_heights = load_data(n_k_m_datapath, m_h_datapath)

In [11]:
print(f'{n_k_m_P[0:3]}')
print(f'{m_heights[0:3]}')

[[9, 4, 2, array([[ 1, -2,  2, -1,  4],
       [ 3,  0, -2,  1,  3],
       [ 0,  4,  4,  4,  3],
       [-3, -1,  4,  1, -4]])], [9, 4, 3, array([[ 1, -2,  2, -1,  4],
       [ 3,  0, -2,  1,  3],
       [ 0,  4,  4,  4,  3],
       [-3, -1,  4,  1, -4]])], [9, 4, 4, array([[ 1, -2,  2, -1,  4],
       [ 3,  0, -2,  1,  3],
       [ 0,  4,  4,  4,  3],
       [-3, -1,  4,  1, -4]])]]
[11.472222222222221, 17.875, 30.0]


In [12]:
# Train one model per (n, k, m) combo using the simplified helper
from model_mheight import SUPPORTED_COMBOS, train_model_for_combo

samples = list(n_k_m_P)
targets = list(m_heights)
metrics_by_combo = {}

for combo in SUPPORTED_COMBOS:
    print(f"Training model for combo {combo}...")
    metrics_by_combo[combo] = train_model_for_combo(
        combo=combo,
        samples=samples,
        targets=targets,
        epochs=200,
        batch_size=512,
        save_dir="trained_models",
    )

metrics_by_combo


Training model for combo (9, 4, 2)...
Epoch 1/200
5/5 - 3s - loss: 3.2947 - val_loss: 2.4470 - 3s/epoch - 688ms/step
Epoch 2/200
5/5 - 0s - loss: 2.4205 - val_loss: 1.8749 - 42ms/epoch - 8ms/step
Epoch 3/200
5/5 - 0s - loss: 1.9623 - val_loss: 1.5802 - 41ms/epoch - 8ms/step
Epoch 4/200
5/5 - 0s - loss: 1.7073 - val_loss: 1.4226 - 47ms/epoch - 9ms/step
Epoch 5/200
5/5 - 0s - loss: 1.5526 - val_loss: 1.3283 - 41ms/epoch - 8ms/step
Epoch 6/200
5/5 - 0s - loss: 1.4561 - val_loss: 1.2626 - 38ms/epoch - 8ms/step
Epoch 7/200
5/5 - 0s - loss: 1.3832 - val_loss: 1.2131 - 41ms/epoch - 8ms/step
Epoch 8/200
5/5 - 0s - loss: 1.3277 - val_loss: 1.1749 - 45ms/epoch - 9ms/step
Epoch 9/200
5/5 - 0s - loss: 1.2778 - val_loss: 1.1450 - 49ms/epoch - 10ms/step
Epoch 10/200
5/5 - 0s - loss: 1.2393 - val_loss: 1.1210 - 40ms/epoch - 8ms/step
Epoch 11/200
5/5 - 0s - loss: 1.2085 - val_loss: 1.1016 - 42ms/epoch - 8ms/step
Epoch 12/200
5/5 - 0s - loss: 1.1818 - val_loss: 1.0861 - 40ms/epoch - 8ms/step
Epoch 13/2

INFO:tensorflow:Assets written to: trained_models/9_4_2_model/assets


Training model for combo (9, 4, 3)...
Epoch 1/200
5/5 - 3s - loss: 3.6493 - val_loss: 3.2768 - 3s/epoch - 685ms/step
Epoch 2/200
5/5 - 0s - loss: 3.0506 - val_loss: 2.7843 - 42ms/epoch - 8ms/step
Epoch 3/200
5/5 - 0s - loss: 2.5882 - val_loss: 2.4351 - 38ms/epoch - 8ms/step
Epoch 4/200
5/5 - 0s - loss: 2.2843 - val_loss: 2.1865 - 39ms/epoch - 8ms/step
Epoch 5/200
5/5 - 0s - loss: 2.0656 - val_loss: 2.0050 - 42ms/epoch - 8ms/step
Epoch 6/200
5/5 - 0s - loss: 1.9082 - val_loss: 1.8684 - 43ms/epoch - 9ms/step
Epoch 7/200
5/5 - 0s - loss: 1.7903 - val_loss: 1.7586 - 38ms/epoch - 8ms/step
Epoch 8/200
5/5 - 0s - loss: 1.6948 - val_loss: 1.6657 - 46ms/epoch - 9ms/step
Epoch 9/200
5/5 - 0s - loss: 1.6110 - val_loss: 1.5851 - 37ms/epoch - 7ms/step
Epoch 10/200
5/5 - 0s - loss: 1.5434 - val_loss: 1.5143 - 38ms/epoch - 8ms/step
Epoch 11/200
5/5 - 0s - loss: 1.4825 - val_loss: 1.4510 - 40ms/epoch - 8ms/step
Epoch 12/200
5/5 - 0s - loss: 1.4295 - val_loss: 1.3942 - 38ms/epoch - 8ms/step
Epoch 13/20

INFO:tensorflow:Assets written to: trained_models/9_4_3_model/assets


Training model for combo (9, 4, 4)...
Epoch 1/200
4/4 - 3s - loss: 3.9228 - val_loss: 3.7755 - 3s/epoch - 856ms/step
Epoch 2/200
4/4 - 0s - loss: 3.7243 - val_loss: 3.5755 - 34ms/epoch - 8ms/step
Epoch 3/200
4/4 - 0s - loss: 3.5365 - val_loss: 3.4102 - 38ms/epoch - 10ms/step
Epoch 4/200
4/4 - 0s - loss: 3.3828 - val_loss: 3.2844 - 33ms/epoch - 8ms/step
Epoch 5/200
4/4 - 0s - loss: 3.2637 - val_loss: 3.1853 - 34ms/epoch - 9ms/step
Epoch 6/200
4/4 - 0s - loss: 3.1696 - val_loss: 3.0984 - 34ms/epoch - 9ms/step
Epoch 7/200
4/4 - 0s - loss: 3.0863 - val_loss: 3.0173 - 35ms/epoch - 9ms/step
Epoch 8/200
4/4 - 0s - loss: 3.0091 - val_loss: 2.9401 - 37ms/epoch - 9ms/step
Epoch 9/200
4/4 - 0s - loss: 2.9323 - val_loss: 2.8663 - 35ms/epoch - 9ms/step
Epoch 10/200
4/4 - 0s - loss: 2.8593 - val_loss: 2.7946 - 37ms/epoch - 9ms/step
Epoch 11/200
4/4 - 0s - loss: 2.7938 - val_loss: 2.7241 - 34ms/epoch - 8ms/step
Epoch 12/200
4/4 - 0s - loss: 2.7264 - val_loss: 2.6550 - 34ms/epoch - 9ms/step
Epoch 13/2

...

Training model for combo (9, 6, 2)...
Epoch 1/200
16/16 - 4s - loss: 3.5941 - val_loss: 2.9121 - 4s/epoch - 240ms/step
Epoch 2/200
16/16 - 0s - loss: 2.6639 - val_loss: 2.2539 - 87ms/epoch - 5ms/step
Epoch 3/200
16/16 - 0s - loss: 2.1579 - val_loss: 1.8805 - 85ms/epoch - 5ms/step
Epoch 4/200
16/16 - 0s - loss: 1.8163 - val_loss: 1.6091 - 84ms/epoch - 5ms/step
Epoch 5/200
16/16 - 0s - loss: 1.5700 - val_loss: 1.4236 - 81ms/epoch - 5ms/step
Epoch 6/200
16/16 - 0s - loss: 1.4014 - val_loss: 1.3012 - 99ms/epoch - 6ms/step
Epoch 7/200
16/16 - 0s - loss: 1.2879 - val_loss: 1.2198 - 85ms/epoch - 5ms/step
Epoch 8/200
16/16 - 0s - loss: 1.2127 - val_loss: 1.1644 - 93ms/epoch - 6ms/step
Epoch 9/200
16/16 - 0s - loss: 1.1599 - val_loss: 1.1271 - 93ms/epoch - 6ms/step
Epoch 10/200
16/16 - 0s - loss: 1.1236 - val_loss: 1.1018 - 88ms/epoch - 6ms/step
Epoch 11/200
16/16 - 0s - loss: 1.0969 - val_loss: 1.0856 - 94ms/epoch - 6ms/step
Epoch 12/200
16/16 - 0s - loss: 1.0801 - val_loss: 1.0749 - 88ms/epoc

INFO:tensorflow:Assets written to: trained_models/9_6_2_model/assets


Training model for combo (9, 6, 3)...
Epoch 1/200
8/8 - 3s - loss: 3.9789 - val_loss: 3.9450 - 3s/epoch - 431ms/step
Epoch 2/200
8/8 - 0s - loss: 3.9313 - val_loss: 3.8931 - 59ms/epoch - 7ms/step
Epoch 3/200
8/8 - 0s - loss: 3.8840 - val_loss: 3.8504 - 51ms/epoch - 6ms/step
Epoch 4/200
8/8 - 0s - loss: 3.8441 - val_loss: 3.8104 - 48ms/epoch - 6ms/step
Epoch 5/200
8/8 - 0s - loss: 3.8038 - val_loss: 3.7669 - 51ms/epoch - 6ms/step
Epoch 6/200
8/8 - 0s - loss: 3.7589 - val_loss: 3.7165 - 48ms/epoch - 6ms/step
Epoch 7/200
8/8 - 0s - loss: 3.7071 - val_loss: 3.6615 - 50ms/epoch - 6ms/step
Epoch 8/200
8/8 - 0s - loss: 3.6519 - val_loss: 3.6013 - 51ms/epoch - 6ms/step
Epoch 9/200
8/8 - 0s - loss: 3.5902 - val_loss: 3.5364 - 49ms/epoch - 6ms/step
Epoch 10/200
8/8 - 0s - loss: 3.5252 - val_loss: 3.4679 - 59ms/epoch - 7ms/step
Epoch 11/200
8/8 - 0s - loss: 3.4557 - val_loss: 3.3957 - 54ms/epoch - 7ms/step
Epoch 12/200
8/8 - 0s - loss: 3.3849 - val_loss: 3.3207 - 53ms/epoch - 7ms/step
Epoch 13/20

INFO:tensorflow:Assets written to: trained_models/9_6_3_model/assets


{(9, 4, 2): {'train_loss': 0.11490057408809662,
  'val_loss': 0.14337849617004395,
  'test_loss': 0.14508314430713654,
  'model_path': 'trained_models/9_4_2_model'},
 (9, 4, 3): {'train_loss': 0.12617774307727814,
  'val_loss': 0.12286161631345749,
  'test_loss': 0.14592625200748444,
  'model_path': 'trained_models/9_4_3_model'},
 (9, 4, 4): {'train_loss': 0.31833362579345703,
  'val_loss': 0.3603180944919586,
  'test_loss': 0.3534606099128723,
  'model_path': 'trained_models/9_4_4_model'},
 (9, 4, 5): {'train_loss': 1.2578506469726562,
  'val_loss': 1.1996803283691406,
  'test_loss': 1.5003057718276978,
  'model_path': 'trained_models/9_4_5_model'},
 (9, 5, 2): {'train_loss': 0.09761608392000198,
  'val_loss': 0.14618739485740662,
  'test_loss': 0.15937012434005737,
  'model_path': 'trained_models/9_5_2_model'},
 (9, 5, 3): {'train_loss': 0.27950769662857056,
  'val_loss': 0.3560623824596405,
  'test_loss': 0.33088168501853943,
  'model_path': 'trained_models/9_5_3_model'},
 (9, 5, 4)